In [ ]:
import numpy as np
import pandas as pd
import urllib
import os
from bs4 import BeautifulSoup
import re
import pickle
import nltk
import string
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns

from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from collections import Counter
from PIL import Image
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

In [ ]:
def loadFile(name):
    directory = str(os.getcwd())
    filepath = os.path.join(directory, name)
    with open(filepath,'r') as f:
        data = f.readlines()
    data = list(set(data))
    result = []
    for d in data:
        d = str(urllib.parse.unquote(d))
        result.append(d)
    return result

In [ ]:
badQueries = loadFile("badqueries.txt")
goodQueries = loadFile("goodqueries.txt")

In [ ]:
bad_df = pd.DataFrame(badQueries)
good_df = pd.DataFrame(goodQueries)

In [ ]:
bad_df.columns = ["query"]
good_df.columns = ["query"]

In [ ]:
bad_df.head()

In [ ]:
good_df.head()

In [ ]:
text = good_df.loc[0].str
text = text[:-2]
text

In [ ]:
def clean_newline(column):
    column[:-2]
    return column[:-2]

In [ ]:
good_df["query"] = good_df["query"].apply(clean_newline)

In [ ]:
good_df.head()

In [ ]:
bad_df["query"] = bad_df["query"].apply(clean_newline)

In [ ]:
bad_df.head()

In [ ]:
good_df["label"] = 0
bad_df["label"] = 1

In [ ]:
good_df.shape[0] + bad_df.shape[0]

In [ ]:
df = pd.concat([good_df, bad_df], axis=0)

In [ ]:
df.shape[0]

In [ ]:
df.head()

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df.head()

In [ ]:
df["label"].value_counts().plot(kind="pie", autopct="%.2f%%").set_title("MALICIOUS - BENIGN")

In [ ]:
def clear_first_char(column):
    if column.startswith("/"):
        return column[1:]
    else:
        return column

In [ ]:
df["query"] = df["query"].apply(clear_first_char)

In [ ]:
df.head()

# Preprocess

In [ ]:
def xss_check(input_string):
    xss_pattern = re.compile(r'(<|>|&lt;|&gt;|script|alert|document\.|onload\=|onerror\=|eval\(|expression\(|prompt\(|confirm\()')
    if xss_pattern.search(input_string.split("/")[-1]):
        return 1
    else:
        return 0

In [ ]:
df["is_xss"] = df["query"].apply(xss_check)

In [ ]:
df[df["is_xss"] == 1].head()

In [ ]:
def lfi_check(input_string):
    lfi_pattern = re.compile(r'(file\:\/\/|(\.\.\/)|(\.\.\\))')
    if "=" in input_string.split("/")[-1]:
        if lfi_pattern.search(input_string.split("/")[-1].split("=", 1)[1]):
            return 1
        else:
            return 0
    elif lfi_pattern.search(input_string.split("/")[-1]):
        return 1
    else:
        return 0

In [ ]:
"scripts/misc/audio.php?recording=../version.in".split("=", 1)

In [ ]:
df["is_lfi"] = df["query"].apply(lfi_check)

In [ ]:
df[df["is_lfi"] == 1].sample(5)

In [ ]:
def command_injection_check(input_string):
    cmd_injection_pattern = re.compile(r'(;|\||`|\$\(|\$\{)')

    if cmd_injection_pattern.search(input_string):
        return 1
    else:
        return 0

In [ ]:
df["is_oci"] = df["query"].apply(command_injection_check)

In [ ]:
df[df["is_oci"] == 1].sample(5)

In [ ]:
def sql_injection_check(input_string):
    sql_injection_pattern = re.compile(r'(\b(SELECT|INSERT|UPDATE|DELETE|FROM|WHERE|AND|OR|UNION|ALL|EXEC|EXECUTE|DECLARE|CAST)\b)')

    if sql_injection_pattern.search(input_string):
        return 1
    else:
        return 0

In [ ]:
df["is_sqli"] = df["query"].apply(sql_injection_check)

In [ ]:
df[df["is_sqli"] == 1].sample(5)

In [ ]:
def urllen(url):
    return len(url)

In [ ]:
df["url_len"] = df["query"].apply(urllen)

In [ ]:
df.head()

In [ ]:
def delimitercount(url):
    return url.count(";") + url.count("_") + url.count("?") + url.count("=") + url.count("&") + url.count("|")

In [ ]:
df["delim_count"] = df["query"].apply(delimitercount)

In [ ]:
df.head()

In [ ]:
def dotcount(url):
    return url.count(".")

In [ ]:
df["dot_count"] = df["query"].apply(dotcount)

In [ ]:
df.head()

In [ ]:
def atcount(url):
    return url.count("@")

In [ ]:
df["at_count"] = df["query"].apply(atcount)

In [ ]:
df[df["at_count"] == 1].head()

In [ ]:
df.sample(5)

In [ ]:
def subdircount(url):
    return url.count("/")

In [ ]:
df["subdir_count"] = df["query"].apply(subdircount)

In [ ]:
df.head()

In [ ]:
def query_count(query):
    if "=" in query:
        return len(query.split("=")[1])
    else:
        return 0

In [ ]:
df["query_len"] = df["query"].apply(query_count)

In [ ]:
df[df["query_len"] != 0].head()

In [ ]:
def total_digits_in_url(url):
    total_digits = 0
    for text in list(map(str, "0123456789")):
        total_digits += url.lower().count(text)
        
    return total_digits

In [ ]:
df["total_digits_url"] = df["query"].apply(total_digits_in_url)

In [ ]:
df.sample(5)

In [ ]:
def total_letter_in_url(url):
    total_letter = 0
    for text in url:
        if text not in "0123456789":
            if text not in string.punctuation:
                total_letter += 1
        
    return total_letter

In [ ]:
df["total_letter_url"] = df["query"].apply(total_letter_in_url)

In [ ]:
string.punctuation

In [ ]:
text = "adas!!1231.@"
total_letter_in_url(text)

In [ ]:
url = "/text/shell.php?acas"
url.split("/")[-1]

In [ ]:
tokenizer = RegexpTokenizer(r"[A-Za-z]+")

In [ ]:
df["url_tokenized"] = df["query"].apply(lambda x: tokenizer.tokenize(x))

In [ ]:
df.sample(5)

In [ ]:
stemmer = SnowballStemmer("english")

In [ ]:
def stem_url(column):
    words = [stemmer.stem(word) for word in column]
    return " ".join(words)

In [ ]:
df["url_stemmed"] = df["url_tokenized"].apply(stem_url)

In [ ]:
df.sample(5)

In [ ]:
cn_phishing = Counter()
cn_safe = Counter()

for text, phishing in zip(df.url_stemmed.values, df.label.values):
    for word in text.split():
        if len(word) >= 4:
            if phishing == 1:
                cn_phishing[word] += 1
            else:
                cn_safe[word] += 1
            
phishing_most_common_words = cn_phishing.most_common(30)
safe_most_common_words = cn_safe.most_common(30)

In [ ]:
def word_freq(freq_top, LABEL):
    words = [word for word, _ in freq_top]
    counts = [count for _, count in freq_top]
    
    plt.figure(figsize=(20, 5))
    ax = sns.barplot(x=words, y=counts)
    plt.title(f"TOP 20 WORDS IN - {'BENIGN' if LABEL == 0 else 'MALICIOUS'}")
    plt.ylabel("Frequency")
    plt.xlabel("Words")
    plt.show()
    
    return freq_top

In [ ]:
benign_top = word_freq(safe_most_common_words, 0)

In [ ]:
malicious_top = word_freq(phishing_most_common_words, 1)

In [ ]:
def print_wordcloud(dict_top, LABEL, mask):
    dict_top = dict(dict_top)
    wordcloud = WordCloud(width=350, height=350, mask=np.array(Image.open(mask)), background_color="white").generate_from_frequencies(dict_top)
    
    plt.figure()
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad=0)
    plt.title(f"TOP 20 WORDS IN - {'BENIGN' if LABEL == 0 else 'MALICIOUS'}")
    plt.show()

In [ ]:
print_wordcloud(benign_top, 0, mask="../masks-wordclouds/user.png")

In [ ]:
print_wordcloud(malicious_top, 1, mask="../masks-wordclouds/star.png")

In [ ]:
df.sample(10)

In [ ]:
def total_digits_domain(url):
    if "/" in url:
        path = url.split("/")[-1]
        dom = url.replace(path, "")
        return total_digits_in_url(dom)
    else:
        return 0

In [ ]:
df["total_digits_domain"] = df["query"].apply(total_digits_domain)

In [ ]:
def total_letter_domain(url):
    if "/" in url:
        path = url.split("/")[-1]
        dom = url.replace(path, "")
        return total_letter_in_url(dom)
    else:
        return 0

In [ ]:
df["total_letter_domain"] = df["query"].apply(total_letter_domain)

In [ ]:
def total_digits_path(url):
    if "/" in url:
        path = url.split("/")[-1]
        return total_digits_in_url(path)
    else:
        return total_digits_in_url(url)

In [ ]:
df["total_digits_path"] = df["query"].apply(total_digits_path)

In [ ]:
def total_letter_path(url):
    if "/" in url:
        path = url.split("/")[-1]
        return total_letter_in_url(path)
    else:
        return total_letter_in_url(url)

In [ ]:
df["total_letter_path"] = df["query"].apply(total_letter_path)

In [ ]:
df.sample(10)

In [ ]:
def get_histplot_central_tendency(df, field, target):
    v_dist_1 = df[field].values
    plt.figure()
    sns.histplot(v_dist_1, kde=True)

    mean = df[field].mean()
    median = df[field].median()
    mode = df[field].mode()

    plt.axvline(mean, color="r", linestyle="--", label="Mean")
    plt.axvline(median, color="g", linestyle="-", label="Median")
    plt.axvline(median, color="b", linestyle=":", label="Mode")
    plt.xlabel(f"count", fontsize=13, color="#333F4B")
    plt.ylabel(f"{field}", fontsize=13, color="#333F4B")
    plt.legend()
    plt.grid(False)
    plt.title(f"Representation Histogram for {field} - {target}", fontsize=18)
    plt.plot(color="white", lw=3)
    plt.show()

In [ ]:
def has_extension(column):
    if "." in column.split("/")[-1]:
        return 1
    else:
        return 0

In [ ]:
df["has_extension"] = df["query"].apply(has_extension)

In [ ]:
df[df["has_extension"] == 1].sample(5)

In [ ]:
def find_extension(column):
    text = column.split("/")[-1]
    if "." in text:
        if "?" in text:
            return text.split("?", 1)[0]
        else:
            return text.split(".", 1)[1]
    else:
        return ""

In [ ]:
df["extension"] = df["query"].apply(find_extension)

In [ ]:
df[df["has_extension"] == 1].sample(5)

In [ ]:
cn_phishing_extension = Counter()
cn_safe_extension = Counter()

for text, phishing in zip(df.extension.values, df.label.values):
    for word in text.split():
        if len(word) != "":
            if phishing == 1:
                cn_phishing_extension[word] += 1
            else:
                cn_safe_extension[word] += 1
            
phishing_most_common_extensions = cn_phishing_extension.most_common(30)
safe_most_common_extensions = cn_safe_extension.most_common(30)

In [ ]:
phishing_extension_top = word_freq(phishing_most_common_extensions, 1)

In [ ]:
safe_extension_top = word_freq(safe_most_common_extensions, 0)

In [ ]:
def has_parameter(column):
    if "?" in column.split("/")[-1]:
        return 1
    else:
        return 0

In [ ]:
df["has_parameter"] = df["query"].apply(has_parameter)

In [ ]:
df[df["has_parameter"] == 1].sample(5)

In [ ]:
df[df["has_parameter"] == 1].sample(1).T

In [ ]:
df.columns

In [ ]:
features = ['is_xss', 'is_lfi', 'is_oci', 'is_sqli', 'url_len',
       'delim_count', 'dot_count', 'at_count', 'subdir_count', 'query_len',
       'total_digits_url', 'total_letter_url', 'total_digits_domain', 
       'total_letter_domain', 'total_digits_path',
       'total_letter_path', 'has_extension', 'has_parameter']

In [ ]:
for feature in features:
    get_histplot_central_tendency(df[df["label"] == 0], feature, "Good URLs")

In [ ]:
for feature in features:
    get_histplot_central_tendency(df[df["label"] == 1], feature, "Bad URLs")

In [ ]:
df.head()

# Model

In [ ]:
X = df.drop(["query", "label", "url_tokenized", "url_stemmed", "extension"], axis=1)
y = df["label"]

In [ ]:
ss = StandardScaler()
X_scaled = ss.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

In [ ]:
pred_test = logreg.predict(X_test)

In [ ]:
pred_train = logreg.predict(X_train)

In [ ]:
train_score = accuracy_score(y_train, pred_train)
print("Train Score: ", train_score)

In [ ]:
test_score = accuracy_score(y_test, pred_test)
print("Test Score:", test_score)

In [ ]:
pickle.dump(logreg, open("waf/website/models/logreg.pkl", "wb"))
pickle.dump(ss, open("waf/website/models/logreg_ss.pkl", "wb"))

In [ ]:
cm = confusion_matrix(pred_test, y_test)
plt.figure()
sns.heatmap(cm, annot=True, cmap="Pastel1", fmt=".0f")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

In [ ]:
df.columns

# Test

In [ ]:
columns = ['query', 'is_xss', 'is_lfi', 'is_oci', 'is_sqli', 'url_len',
       'delim_count', 'dot_count', 'at_count', 'subdir_count', 'query_len',
       'total_digits_url', 'total_letter_url', 'url_tokenized', 'url_stemmed',
       'total_digits_domain', 'total_letter_domain', 'total_digits_path',
       'total_letter_path', 'has_extension', 'extension', 'has_parameter']

In [ ]:
test1 = 'hrttz9fj.dll?<script>document.cookie="testtbjy=7334;"</script>' # 1
test2 = 'index.php?option=com_mailto&tmpl=component&link=aHR0cDovL2FkdmVudHVyZ' # 1
test3 = 'nba/player/_/id/3457/brandon-rush' # 0
test4 = '?q=anthony-hamilton-soulife' # 0
test5 = 'site/relationship_detail.php?name=Martin-Brodeur&celebid=12150&relid=11371' # 0

In [ ]:
test_df = pd.DataFrame(columns=columns)

In [ ]:
test_df.loc[0, "query"] = test1
test_df.loc[1, "query"] = test2
test_df.loc[2, "query"] = test3
test_df.loc[3, "query"] = test4
test_df.loc[4, "query"] = test5

In [ ]:
test_df.head()

In [ ]:
stemmer = SnowballStemmer("english")
tokenizer = RegexpTokenizer(r"[A-Za-z]+")

In [ ]:
def preprocess(test_df):
    stemmer = SnowballStemmer("english")
    tokenizer = RegexpTokenizer(r"[A-Za-z]+")
    test_df["is_xss"] = test_df["query"].apply(xss_check)
    test_df["is_lfi"] = test_df["query"].apply(lfi_check)
    test_df["is_oci"] = test_df["query"].apply(command_injection_check)
    test_df["is_sqli"] = test_df["query"].apply(sql_injection_check)
    test_df["url_len"] = test_df["query"].apply(urllen)
    test_df["delim_count"] = test_df["query"].apply(delimitercount)
    test_df["dot_count"] = test_df["query"].apply(dotcount)
    test_df["at_count"] = test_df["query"].apply(atcount)
    test_df["subdir_count"] = test_df["query"].apply(subdircount)
    test_df["query_len"] = test_df["query"].apply(query_count)
    test_df["total_digits_url"] = test_df["query"].apply(total_digits_in_url)
    test_df["total_letter_url"] = test_df["query"].apply(total_letter_in_url)
    test_df["url_tokenized"] = test_df["query"].apply(lambda x: tokenizer.tokenize(x))
    test_df["url_stemmed"] = test_df["url_tokenized"].apply(stem_url)
    test_df["total_digits_domain"] = test_df["query"].apply(total_digits_domain)
    test_df["total_letter_domain"] = test_df["query"].apply(total_letter_domain)
    test_df["total_digits_path"] = test_df["query"].apply(total_digits_path)
    test_df["total_letter_path"] = test_df["query"].apply(total_letter_path)
    test_df["has_extension"] = test_df["query"].apply(has_extension)
    test_df["extension"] = test_df["query"].apply(find_extension)
    test_df["has_parameter"] = test_df["query"].apply(has_parameter)
    return test_df

In [ ]:
test_df = preprocess(test_df)

In [ ]:
test_df.head()

In [ ]:
dropped = test_df.drop(["query", "url_tokenized", "url_stemmed", "extension"], axis=1)

In [ ]:
dropped_scaled = ss.transform(dropped)

In [ ]:
logreg.predict(dropped_scaled)

In [ ]:
log = pd.